# Aave Queries

In [1]:
# Historical liquidations, size of position and associated tokens
query_TotalLiquidations = """
query TotalLiquidations($txnID: ID) {
  liquidates(first: 1000, where: {id_gt: $txnID}) {
    hash
    amountUSD
    timestamp
    liquidatee {
      id
    }
    asset {
      name
      symbol
    }
  }
}
"""

In [2]:
# Historical deposits, size of position and associated tokens
query_CollateralSuppliedDeposits = """
query CollateralSuppliedDeposits($txnID: ID) {
  deposits (first: 1000, where: {id_gt: $txnID}) {
    hash
    amountUSD
    timestamp
    account {
      id
    }
    asset {
      name
      symbol
    }
  }
}
"""

In [ ]:
# Historical withdraws, size of position and associated tokens
query_CollateralSuppliedWithdraws = """
query CollateralSuppliedWithdraws($txnID: ID) {
  withdraws(first: 1000, where: {id_gt: $txnID}) {
    hash
    amountUSD
    timestamp
    account {
      id
    }
    asset {
      name
      symbol
    }
  }
}

"""

In [3]:
# Historical borrows, size of position and associated tokens
query_Borrows = """
query Borrows($txnID: ID) {
  borrows(first: 1000, where: {id_gt: $txnID}) {
    hash
    amountUSD
    timestamp
    account {
      id
    }
    asset {
      name
      symbol
    }
  }
}

"""

In [4]:
# Historical repays, size of position and associated tokens
query_Repays = """
query Repays($txnID: ID) {
  repays(first: 1000, where: {id_gt: $txnID}) {
    hash
    amountUSD
    timestamp
    account {
      id
    }
    asset {
      name
      symbol
    }
  }
}
"""

In [5]:
# Historical borrows and their associated rate type: variable or stable
query_ratetype = """
query ratetype($txnID: ID) {
  borrows(first: 1000, where: {id_gt: $txnID}) {
    id
    borrowRateMode
  }
}
"""

## Import Libraries

In [11]:
import requests
import pandas as pd
import json
from datetime import datetime,timedelta
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## Data Set Up


In [12]:
# General Format --> {[query] : ([parent_entity], [child_entity], [type_of_data])}

query_pairs = {query_CollateralSuppliedDeposits    : ("deposits","hash", "txn_data"),
               query_CollateralSuppliedWithdraws   : ("withdraws","hash", "txn_data"),
               query_TotalLiquidations             : ("liquidates","hash", "txn_data"),
               query_Borrows                       : ("borrows","hash", "txn_data"),
               query_Repays                        : ("repays","hash", "txn_data"),
               query_ratetype                      : ("borrows", "id", "param_data")}

# General Format --> {([Version], [Market], [type_of_data]) : [Subgraph_Api_Endpoint]}

api_dict = {("V2", "Ethereum", "txn_data")    : "https://api.thegraph.com/subgraphs/name/messari/aave-v2-ethereum",
            ("V2", "Avalanche", "txn_data")   : "https://api.thegraph.com/subgraphs/name/messari/aave-v2-avalanche",
            ("V2", "Ethereum", "param_data")  : "https://api.thegraph.com/subgraphs/name/aave/protocol-v2",
            ("V2", "Avalanche", "param_data") : "https://api.thegraph.com/subgraphs/name/aave/protocol-v2-avalanche"}

# Sidelined for now
# ("V2", "Polygon", "txn_data")     : "https://api.thegraph.com/subgraphs/name/messari/aave-v2-polygon"
# ("V2", "Polygon", "param_data")   : "https://api.thegraph.com/subgraphs/name/aave/aave-v2-matic"

## Create Dataframes to Store Data

In [13]:
query_dict = {}                                                               

for query,attributes in query_pairs.items():
    for (version,market,data) in api_dict.keys():
        if attributes[2] == data:
            query_dict[(version,market,attributes[2],attributes[0],attributes[1],query)] = pd.DataFrame()
        else:
            continue

## Pagination Collection of all Entries from Subgraphs

In [115]:
%%time

entity = 0
count = 1

# Iterate through all queries
for attributes in query_dict.keys():
    
    # Assign last transaction hash
    txnID = {'txnID' : ""}                                                    

    while True:
        
        version, market, query = attributes[0],attributes[1], attributes[5]
        
        # Must update 
        api_endpoint = api_dict[(version,market,attributes[2])]
        request = requests.post(api_endpoint, json={'query': query,'variables': txnID})  
        
        # MUST UPDATE INDEXING LOGIC BASED ON QUERY
        df_test = request.json()
        df_flatten = pd.json_normalize((df_test["data"][attributes[3]]))       
                
        # MUST CREATE NEW DATAFRAME BASED ON QUERY
        query_dict[attributes] = query_dict[attributes].append(df_flatten, ignore_index = True) 
        txnID_upd = {'txnID' : df_flatten[attributes[4]][len(df_flatten)-1]}
        print(query_dict[attributes].shape)
        
        txnID.update(txnID_upd)

        if len(df_flatten) == 1000:
            count += 1
            continue
        else:
            count += 1
            break
    entity = entity + 1
    print(entity)

## *TEMP DOCUMENT IMPORT/EXPORT*

In [24]:
# Export files to store for temporary use --> update file path 
for keys,df in query_dict.items():
    filename = 'PATH_NAME'+str(keys[:5])
    df.to_parquet(filename+".parquet")

In [14]:
# Read files --> update file path 
for keys,df in query_dict.items():
    filename = 'PATH_NAME'+str(keys[:5])+'.parquet'
    try:
        query_dict[keys] = pd.read_parquet(filename)
        print("Success")
    except:
        print(filename)

Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


## One-Off Manipulation Step for Liquidations

In [15]:
# Update column "liquidatee" to "account.id"

# Iterate through all query dataframes
for keys,df in query_dict.items():
    for col in df.columns:
        if col == "liquidatee.id":
            df.rename(columns={"liquidatee.id": "account.id"}, inplace=True)
            
        elif col == "reserve.name" in df.columns:
            df.rename(columns={"reserve.name": "asset.name"}, inplace=True)
            
        elif col == "reserve.symbol" in df.columns:
            df.rename(columns={"reserve.symbol": "asset.symbol"}, inplace=True)
        
        # Ethereum and Avalanche store txn hash differently compared to Polygon
        elif col == 'id' in df.columns:
            df["id"] = df["id"].str.split(":", expand = True)[2]

## Unique Identifier Manipulation Step

In [16]:
%%time

# Iterate through all query dataframes
for keys,df in query_dict.items():

    # Add column "hash_unique"
    df[keys[4]+"_unique"] = ""
    
    # Create dictionary of hashes with 0 as the initial counter value
    current_counter = {}
    for ID in df[keys[4]].value_counts().index:
        current_counter[ID] = 0

    # Append add'l identifier to "hash" in "hash_unique"
    for i,r in df.iterrows():
        r[keys[4]+"_unique"] = r[keys[4]] +"_"+str(len(keys[3]))+str(current_counter[r[keys[4]]])
        current_counter[r[keys[4]]]+=1

    # Drop child_entity column
     # df.drop(keys[4], axis=1, inplace=True)
    

CPU times: user 2min 37s, sys: 1.74 s, total: 2min 38s
Wall time: 2min 42s


## Conversion Steps of Dataframes: Timestamp and amountUSD

In [17]:
%%time

# Iterate through all query dataframes
for keys,df in query_dict.items():

    # Remove all '.e' extensions under symbol
    try:
        df["asset.symbol"] = df["asset.symbol"].str.upper()
        df["asset.symbol"] = df['asset.symbol'].str.replace(".E", "", regex=False)
    except:
        print(keys[:5], "asset.symbol not in dataframe.")
    
    # Convert all timestamps to datetime format
    try:
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    except:
        print(keys[:5], "timestamp not in dataframe.")
    
    # Convert 'amountUSD' to type float
    try:
        df['amountUSD'] = df['amountUSD'].astype(float)
    except:
        print(keys[:5], "amountUSD not in dataframe.")
    
    # Update transaction id to transaction hash
    try:
        df.rename(columns={"id_unique": "hash_unique"}, inplace=True)
    except:
        print(keys[:5], "id_unique not in dataframe.")


('V2', 'Ethereum', 'param_data', 'borrows', 'id') asset.symbol not in dataframe.
('V2', 'Ethereum', 'param_data', 'borrows', 'id') timestamp not in dataframe.
('V2', 'Ethereum', 'param_data', 'borrows', 'id') amountUSD not in dataframe.
('V2', 'Avalanche', 'param_data', 'borrows', 'id') asset.symbol not in dataframe.
('V2', 'Avalanche', 'param_data', 'borrows', 'id') timestamp not in dataframe.
('V2', 'Avalanche', 'param_data', 'borrows', 'id') amountUSD not in dataframe.
CPU times: user 4.32 s, sys: 659 ms, total: 4.97 s
Wall time: 5.54 s


## Add Version, Market, and Event Type to Each Transaction

In [18]:
%%time

for keys,df in query_dict.items():
    
    # Append version type
    df['version'] = keys[0]
    
    # Append market type
    df['market'] = keys[1]
    
    # Append event type
    df['event_type'] = keys[3]

CPU times: user 41.3 ms, sys: 15.6 ms, total: 56.9 ms
Wall time: 80.3 ms


## Import / Create Grades of Tokens (Aave Risk Team Provided) 

In [20]:
# Check how many UST transactions happened and the amount
# Check RENFIL and UST transactions since they are not allowed to be used as collateral
# ADDED: STETH to 2021/09/24 based on risk parameters https://docs.aave.com/risk/v/aave-v2/asset-risk/risk-parameters
# ADDED: ENS TO 2021/09/24 based on risk parameters https://docs.aave.com/risk/v/aave-v2/asset-risk/risk-parameters
# ADDED: WAVAX to 2021/09/24 based on risk parameters https://docs.aave.com/risk/v/aave-v2/asset-risk/risk-parameters
# ADDED BAL TO 2021/01/14

### NEED TO UPDATE RISK OF CVX

token_risk_dict = {("Ethereum", "2021/09/24") :
                  {'AMPL'  : 'B',  'BUSD'  : 'B+', 'DAI'  : 'B+', 'FEI'  : 'B-', 'FRAX' : 'B-', 
                   'GUSD'  : 'B-', 'PAX'   : 'B+', 'RAI'  : 'C+', 'SUSD' : 'B-', 'TUSD' : 'B-',
                   'USDC'  : 'A-', 'USDT'  : 'B+', 'AAVE' : 'B',  'BAL'  : 'B',  'BAT'  : 'B+', 'CRV'  : 'B',
                   'DPI'   : 'C+', 'ENJ'   : 'B+', 'ETH'  : 'A+', 'KNC'  : 'B',  'LINK' : 'A-', 'MANA' : 'B',
                   'MKR'   : 'B',  'REN'   : 'B',  'SNX'  : 'B',  'UNI'  : 'B+', 'WBTC' : 'B',  'WETH' : 'A',
                   'WMATIC': 'B+', 'XSUSHI': 'B-', 'YFI'  : 'B+', 'ZRX'  : 'B+', 'STETH': 'B+', 'ENS'  : 'B',
                   'WAVAX' : 'B',  'RENFIL': '0',  'UST'  : '0',  '1INCH': '0',  'CVX'  : 'B'},
                   
                   ("Ethereum", "2021/08/10") :
                  {'AMPL'  : 'B-', 'BUSD'  : 'B+', 'DAI'  : 'B+', 'GUSD' : 'B-', 'PAX'  : 'B+', 'RAI'  : 'C+',
                   'SUSD'  : 'B-', 'TUSD'  : 'B-', 'USDC' : 'A-', 'USDT' : 'B+', 'AAVE' : 'B',  'BAL'  : 'B',
                   'BAT'   : 'B+', 'CRV'   : 'B',  'ENJ'  : 'B+', 'KNC'  : 'B',  'LINK' : 'A-', 'MANA' : 'B',
                   'MKR'   : 'B-', 'REN'   : 'B',  'SNX'  : 'B',  'UNI'  : 'B',  'WBTC' : 'B',  'WETH' : 'A-', 
                   'XSUSHI': 'B-', 'YFI'   : 'B',  'ZRX'  : 'B+', 'RENFIL':'0',  'UST'  : '0',  '1INCH': '0',
                   'DPI'   : 'C+'},
                   
                   ("Ethereum", "2021/02/24") : 
                  {'BUSD'  : 'B',  'DAI'   : 'B+', 'GUSD' : 'B-', 'SUSD' : 'B-', 'TUSD' : 'B',  'USDC' : 'A-', 
                   'USDT'  : 'B+', 'AAVE'  : 'B',  'BAL'  : 'B',  'BAT'  : 'B+', 'CRV'  : 'B',  'ENJ'  : 'B+', 
                   'KNC'   : 'B+', 'LINK'  : 'B+', 'MANA' : 'B-', 'MKR'  : 'B-', 'REN'  : 'B',  'SNX'  : 'C+', 
                   'UNI'   : 'B',  'WBTC'  : 'B',  'WETH' : 'A-', 'XSUSHI':'B-', 'YFI'  : 'B',  'ZRX'  : 'B+',
                   'RENFIL': '0',  'UST'   : '0',  '1INCH': '0'},
                   
                   ("Ethereum", "2021/01/14") :
                  {'BUSD'  : 'B',  'DAI'   : 'B+', 'GUSD' : 'B-', 'SUSD' : 'B-', 'TUSD' : 'B',  'USDC': 'A-', 
                   'USDT'  : 'B+', 'AAVE'  : 'B-', 'BAT'  : 'B+', 'CRV'  : 'B-', 'ENJ'  : 'B',  'KNC' : 'B', 
                   'LINK'  : 'B+', 'MANA'  : 'B-', 'MKR'  : 'B-', 'REN'  : 'B',  'SNX'  : 'C+', 'UNI' : 'B', 
                   'WBTC'  : 'B-', 'WETH'  : 'A-', 'YFI'  : 'B',  'ZRX'  : 'B+', 'RENFIL':'0',  'UST' : '0',
                   'BAL'   : 'B'},
                   
                   ("Ethereum", "2020/12/01") :
                  {'BUSD'  : 'B',  'DAI'   : 'B+', 'SUSD' : 'C+', 'TUSD' : 'B',  'USDC' : 'B+', 'USDT' : 'B+',
                   'AAVE'  : 'B-', 'BAT'   : 'B+', 'ENJ'  : 'B+', 'ETH'  : 'A+', 'KNC'  : 'B+', 'LEND' : 'B', 
                   'LINK'  : 'B+', 'MANA'  : 'B-', 'MKR'  : 'B',  'REN'  : 'B',  'REP'  : 'B',  'SNX'  : 'C+', 
                   'UNI'   : 'B',  'WBTC'  : 'B-', 'WETH' : 'B+', 'YFI'  : 'B',  'ZRX'  : 'B+'},
                   
                   ("Ethereum", "2020/10/27"):
                  {'BUSD'  : 'B',  'DAI'   : 'B',  'SUSD' : 'C+', 'TUSD' : 'B',  'USDC' : 'B+', 'USDT' : 'B+', 
                   'AAVE'  : 'C+', 'BAT'   : 'B+', 'ENJ'  : 'B+', 'ETH'  : 'A+', 'KNC'  : 'B+', 'LEND' : 'B', 
                   'LINK'  : 'B+', 'MANA'  : 'B-', 'MKR'  : 'B-', 'REN'  : 'B',  'REP'  : 'B',  'SNX'  : 'C+', 
                   'UNI'   : 'B',  'WBTC'  : 'B-', 'YFI'  : 'B',  'ZRX'  : 'B+'},
                   
                   ("Ethereum", "2020/04/01"):
                  {'DAI'   : 'B-', 'USDC'  : 'B+', 'TUSD' : 'B',  'USDT' : 'B',  'SUSD' : 'C',  'BUSD' : 'B-', 
                   'SNX'   : 'C',  'REP'   : 'B-', 'ZRX'  : 'B+', 'BAT'  : 'B+', 'WBTC' : 'C',  'MKR'  : 'B-', 
                   'LINK'  : 'B+', 'KNC'   : 'B',  'MANA' : 'B-', 'LEND' : 'B',  'ETH'  : 'A',  'SETH' : 'D+'},
                    
                   # AmmWETH, AmmUSDT, AmmGUniDAIUSDC, AmmWBTC risk grades match most recent map above
                   ("AMM", "2021/02/24"):
                  {'AMMBPTBALWETH'  : 'B',  'AMMUNIUSDCWETH' : 'A-', 'AMMWETH'        : 'A',  'AMMUNIBATWETH' : 'A-',
                   'AMMBPTWBTCWETH' : 'B+', 'AMMUSDT'        : 'B+', 'AMMUNIMKRWETH'  : 'B+', 'AMMUNICRVWETH' : 'B+',
                   'AMMUNIUNIWETH'  : 'B+', 'AMMUNIWBTCUSDC' : 'B+', 'AMMUNIDAIUSDC'  : 'A-', 'AMMUNIAAVEWETH': 'B+',
                   'AMMGUNIDAIUSDC' : 'A-', 'AMMUNILINKWETH' : 'A-', 'AMMUNIWBTCWETH' : 'B+', 'AMMWBTC'       : 'B',
                   'AMMUNIYFIWETH'  : 'B+', 'AMMUNIRENWETH'  : 'B+', 'AMMUNIDAIWETH'  : 'A-', 'AMMUNISNXWETH' : 'B'}}

## Creating Raw Database

In [29]:
%%time

# Collect all unique columns from queried data
raw_db_columns = []

for keys,df in query_dict.items():
    temp_columns = list(df.columns)
    for col in temp_columns:
        if col in raw_db_columns:
            continue
        else:
            raw_db_columns.append(col)

# Create the Raw Database
Raw_Database = pd.DataFrame(columns=raw_db_columns)
Raw_Database.drop('borrowRateMode', axis=1, inplace=True)

# Join all dataframes into Raw_Database
for keys,df in query_dict.items():
    if keys[2] == "txn_data":
        Raw_Database = pd.concat([Raw_Database, df],ignore_index = True)

CPU times: user 13.5 s, sys: 1.13 s, total: 14.7 s
Wall time: 15.6 s


## ***TEMP DOCUMENT IMPORT/EXPORT***
test_df_matched is output of all code run above

In [22]:
# Export files to store for temporary use --> update file path 
for keys,df in query_dict.items():
    try:
        filename = 'PATH_NAME'+str(keys[:5])
        df.to_parquet(filename+".parquet")
    except:
        print(keys[:5], 'not exported.')

In [15]:
# Read files --> update file path 
for keys,df in query_dict.items():
    filename = 'PATH_NAME'+str(keys[:5])+".parquet"
    try:
        query_dict[keys] = pd.read_parquet(filename)
        print("Success")
    except:
        print(filename)

Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


In [51]:
# Export file to store for temporary use --> update file path 
Raw_Database.to_parquet('PATH_NAME'+.parquet")

In [37]:
# Read files --> update file path 
Raw_Database = pd.read_parquet('PATH_NAME'+.parquet")


## Match Grade to Each Transaction


In [38]:
%%time

# The minimum difference of the date at which the transaction happened and the time of rating updates will 
# determine the rating. Assumption is the closer to the rating change is a better representation of the level of 
# risk at the time of the transaction.

# Stores subtset of timestamp in new row with standard ISO format
Raw_Database['row_date'] = Raw_Database["timestamp"].astype('str').str[:10]
Raw_Database['row_date'] = pd.to_datetime(Raw_Database['row_date'])

date_list = []

# Calculates difference in time from each ranking of token risk
for date in list(token_risk_dict.keys()):
    Raw_Database[date] = abs(Raw_Database['row_date'] - datetime.strptime(date[1],"%Y/%m/%d"))

Raw_Database['Min_Date'] = Raw_Database[[('Ethereum', '2021/09/24'),
       ('Ethereum', '2021/08/10'), ('Ethereum', '2021/02/24'),
       ('Ethereum', '2021/01/14'), ('Ethereum', '2020/12/01'),
       ('Ethereum', '2020/10/27'), ('Ethereum', '2020/04/01'),
            ('AMM', '2021/02/24')]].idxmin(axis=1)

# Maps token risk with associated transaction
Raw_Database['Min_Date'] = Raw_Database['Min_Date'].map(token_risk_dict)

def get_sublist(sta,end):
    return end[sta]

Raw_Database['token_risk'] = Raw_Database.apply(lambda x: get_sublist(x['asset.symbol'], x['Min_Date']), axis=1)

CPU times: user 2min 13s, sys: 12.5 s, total: 2min 26s
Wall time: 2min 37s


## Match Borrow Transaction Type

In [52]:
%%time

# Merges borrow rate type with the transaction type - a nuance of pulling from Messari and Aave Subgraphs
for borrow_type in query_dict:
    if borrow_type[2] == 'param_data':
        Raw_Database = Raw_Database.merge(query_dict[borrow_type][['borrowRateMode','hash_unique','event_type', 'market']], 
                                          how="left", on=['hash_unique','event_type', 'market'])
        
Raw_Database['borrowRateMode_x'].fillna(Raw_Database['borrowRateMode_y'], inplace=True)
Raw_Database.drop('borrowRateMode_y', axis=1, inplace=True)
Raw_Database.rename(columns={"borrowRateMode_x": "borrowRateMode"}, inplace=True)

CPU times: user 5.18 s, sys: 1.1 s, total: 6.29 s
Wall time: 6.57 s


In [67]:
%%time

# Total number of borrows with no borrow type value attached
borrow_type_count = len(Raw_Database[(Raw_Database.event_type == 'borrows') & (Raw_Database['borrowRateMode'] == 'nan')]['hash_unique'].to_list())

if borrow_type_count == 0:
    print('There are no borrows without a borrow type.')
else:
    print('The number of borrows with no borrow type is', borrow_type_count,'.')

There are no borrows without a borrow type.
CPU times: user 176 ms, sys: 11.6 ms, total: 188 ms
Wall time: 194 ms



## Duration Between Last Transaction Calculation Step

In [56]:
%%time

days = pd.to_timedelta('00:00:00')
Raw_Database.sort_values(['account.id','timestamp'], ascending=[False,True], inplace=True)
Raw_Database['time_btw_txn'] = Raw_Database['timestamp'] - Raw_Database['timestamp'].shift(1)
first_txns = Raw_Database.sort_values(['account.id','timestamp'], ascending=[False,True]).groupby('account.id').head(1)
Raw_Database['time_btw_txn'].mask(Raw_Database['hash_unique'].isin(first_txns['hash_unique']), other=days,inplace=True)


CPU times: user 4.17 s, sys: 1.63 s, total: 5.79 s
Wall time: 6.8 s


## Clean Up Raw_Database

In [57]:
%%time

try:
    # remove all columns where all values are nan/null/none
    Raw_Database.dropna(axis=1, how='all', inplace=True)

except:
    print('All columns have values.')
    
try:
    Raw_Database.drop([('Ethereum', '2021/09/24'), ('Ethereum', '2021/08/10'), ('Ethereum', '2021/02/24'), 
                    ('Ethereum', '2021/01/14'), ('Ethereum', '2020/12/01'), ('Ethereum', '2020/10/27'), 
                    ('Ethereum', '2020/04/01'), ('AMM', '2021/02/24'),'Min_Date','row_date'],
                     axis = 1, inplace=True)

except:
    print('Some columns are not in Raw_Database.')

CPU times: user 3.97 s, sys: 374 ms, total: 4.35 s
Wall time: 4.53 s


## ***TEMP DOCUMENT IMPORT/EXPORT***
test_df_matched is output of all code run above

In [71]:
# Export file to store for temporary use --> update file path 
Raw_Database.to_parquet('PATH_NAME'+'.parquet")

In [104]:
# Read files --> update file path 
Raw_Database = pd.read_parquet('PATH_NAME'+'.parquet')

## Raw_Database Calculations / Derivations

In [105]:
%%time

# Type of Collateral Borrow (A+ = 12, D- = 1)
risk_int_mapping = {"A+" : 1.0, "A" : 2.0, "A-" : 3.0, "B+" : 4.0, "B" : 5.0, "B-" : 6.0,
                    "C+" : 7.0, "C" : 8.0, "C-" : 9.0, "D+" : 10.0,"D" : 11.0,"D-" : 12.0, '0' : 0}
# Convert risk categories to integer
Raw_Database["token_risk_value"] = Raw_Database['token_risk'].replace(risk_int_mapping)

# Borrow risk value calculation
Raw_Database['txn_risk_value'] = Raw_Database['amountUSD'] * Raw_Database['token_risk_value']


CPU times: user 990 ms, sys: 107 ms, total: 1.1 s
Wall time: 1.1 s


## Log Transform of AmountUSD

In [106]:
# Determined from EDA. Will Require additional review as transaction volumne increase with time
Raw_Database['txn_risk_value'] = np.log(Raw_Database['txn_risk_value']).clip(lower=0)

## Create Final Database

In [107]:
%%time

# Create Final Database
Final_Database = pd.DataFrame(Raw_Database['account.id'].drop_duplicates().copy(deep=True).reset_index(drop=True))

CPU times: user 62.9 ms, sys: 38.6 ms, total: 101 ms
Wall time: 130 ms


## Rolling Average Function

In [108]:
## Inputs:
# event_type --> Event is found as the child entity in the query - ONLY FOR TXN_DATA 

def attr_derivations(event_type):
    
    global Final_Database
    
    # Loop through days for each day interval - NEED TO ADD SINCE ADDRESS INCEPTION
    day_interval = [1,7,30,60,90,180,360,720]

    # Parameters
    #today = datetime.today()
    today = datetime(2022,8,17, hour=0, minute=0, second=0, microsecond=0)

    # Create attribute subset
    df_subset = Raw_Database[Raw_Database.event_type == event_type]
                     
    # Sort values by account.id and then timestamp
    df_subset.sort_values(['account.id','timestamp'], ascending=[False,True], inplace=True)
    
    for interval in day_interval:
        
        # Subtract today from each txn date: today_minus_txndate
        df_subset_copy = df_subset.copy(deep=True)
        df_subset_copy['today_minus' + str(interval)] = today - df_subset['timestamp']
                     
        # Create a subset of data that only has timestamp difference values ≤ that interval in day_interval
        df_subset_copy = df_subset_copy[df_subset_copy['today_minus' + str(interval)] <= pd.Timedelta(str(interval)+" days")]
        
        # ROLLING TRANSACTION RISK {deposits, borrows, liquidates, withdrawals, repays}
        df_roll_final = df_subset_copy.groupby('account.id')['txn_risk_value'].sum() / float(interval)
        df_roll_final = df_roll_final.reset_index()
        df_roll_final.rename(columns = {'txn_risk_value' : 'weighted_rolling_'+event_type+str(interval)+'day'}, inplace = True)
        
        # AVERAGE NUMBER OF EVENT TYPES {deposits, borrows, liquidates, withdrawals, repays}
        avg_event_type_final = df_subset_copy.groupby('account.id')['event_type'].count() / float(interval)
        avg_event_type_final = avg_event_type_final.reset_index()
        avg_event_type_final.rename(columns = {'event_type' : 'weighted_count_'+event_type+str(interval)+'day'}, inplace = True)
        
        # Merge on Final_Database
        Final_Database = Final_Database.merge(df_roll_final, on=['account.id'], how='left')
        Final_Database = Final_Database.merge(avg_event_type_final, on=['account.id'], how='left')
    
    
    # AVERAGE EVENT ACTIVITY DORMANCY
    columns = ['weighted_activity_1day', 'weighted_activity_activity7day',
       'weighted_activity_30day', 'weighted_activity_60day',
       'weighted_activity_90day', 'weighted_activity_180day',
       'weighted_activity_360day', 'weighted_activity_720day']
    if any(column in columns for column in Final_Database.columns):
        pass
    else:
        Raw_Database.sort_values(['account.id','timestamp'], ascending=[False,True], inplace=True)
    
        for interval in day_interval:
            
            # Subtract today from each txn date: today_minus_txndate
            Raw_Database_copy = Raw_Database.copy(deep=True)
            Raw_Database_copy['today_minus' + str(interval)] = today - df_subset['timestamp']
            
            # Create a subset of data that only has timestamp difference values ≤ that interval in day_interval
            Raw_Database_copy = Raw_Database_copy[Raw_Database_copy['today_minus' + str(interval)] <= pd.Timedelta(str(interval)+" days")]
            
            # ROLLING TRANSACTION RISK {deposits, borrows, liquidates, withdrawals, repays}
            df_roll_final = Raw_Database_copy.groupby('account.id')['txn_risk_value'].sum() / float(interval)
            df_roll_final = df_roll_final.reset_index()
            df_roll_final.rename(columns = {'txn_risk_value' : 'weighted_activity_'+str(interval)+'day'}, inplace = True)
            
            # Merge on Final_Database
            Final_Database = Final_Database.merge(df_roll_final, on=['account.id'], how='left')

In [109]:
%%time

# Iterating through each event type. Can be found in query_pairs
event_type_list = [*set([item[2] for item in list(api_dict.keys())])]

for event in event_type_list:
    attr_derivations(event)

CPU times: user 17.7 s, sys: 4.56 s, total: 22.3 s
Wall time: 23.6 s


## % of Variable vs Stable Rate Loans
Weighted average number of loans that were stable

In [110]:
%%time

# Total sum of transactional risk value
total_risk_subset = Raw_Database.groupby('account.id')['txn_risk_value'].sum()

# Subset of type variable borrow transactional risk value
var_percent = Raw_Database[Raw_Database['borrowRateMode'] == 'Variable'].groupby('account.id')['txn_risk_value'].sum() / total_risk_subset
var_percent = var_percent.reset_index()
var_percent.rename(columns = {'txn_risk_value' : 'var_percent'}, inplace = True)
Final_Database = Final_Database.merge(var_percent, on=['account.id'], how='left')

# Subset of type variable stable transactional risk value
stable_percent = Raw_Database[Raw_Database['borrowRateMode'] == 'Stable'].groupby('account.id')['txn_risk_value'].sum() / total_risk_subset
stable_percent = stable_percent.reset_index()
stable_percent.rename(columns = {'txn_risk_value' : 'stable_percent'}, inplace = True)
Final_Database = Final_Database.merge(stable_percent, on=['account.id'], how='left')


CPU times: user 795 ms, sys: 138 ms, total: 933 ms
Wall time: 962 ms


## % of Transactions on Each Market
Weighted proportion of transactions on each market

In [111]:
%%time

# Iterating through each event type. Can be found in query_pairs
markets_list = [*set([item[1] for item in list(api_dict.keys())])]

# Total sum of transactional risk value
total_risk_subset = Raw_Database.groupby('account.id')['txn_risk_value'].sum()

# Iterations of each market subset's transactional risk value
for market in markets_list:
    market_percent = Raw_Database[Raw_Database['market'] == market].groupby('account.id')['txn_risk_value'].sum() / total_risk_subset
    market_percent = market_percent.reset_index()
    market_percent.rename(columns = {'txn_risk_value' : str(market)+'_percent'}, inplace = True)
    Final_Database = Final_Database.merge(market_percent, on=['account.id'], how='left')

CPU times: user 1.39 s, sys: 266 ms, total: 1.65 s
Wall time: 1.69 s


## ***TEMP DOCUMENT IMPORT/EXPORT***

In [112]:
# Export file to store for temporary use --> update file path 
Final_Database.to_parquet('PATH_NAME'+'.parquet")

In [113]:
# Read files --> update file path
Final_Database = pd.read_parquet('PATH_NAME'+'.parquet")